In [1]:
import pandas as pd

In [5]:
path = "../../../Dataset/UNSW-NB15/UNSW_NB15_training-set.csv"
dataset = pd.read_csv(path)
dataset["attack_cat"].value_counts()

Normal            37000
Generic           18871
Exploits          11132
Fuzzers            6062
DoS                4089
Reconnaissance     3496
Analysis            677
Backdoor            583
Shellcode           378
Worms                44
Name: attack_cat, dtype: int64